# Model Training and Test

In [10]:
import pickle
import xgboost as xgb
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold, train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score
from sklearn.datasets import load_iris, load_digits, load_boston

In [11]:
with open('./data/train.pkl', mode='rb') as f:
    train = pickle.load(f)
with open('./data/test.pkl', mode='rb') as f:
    test = pickle.load(f)

In [12]:
test.head()

,PassengerId,Pclass,Sex,Age,Fare,Embarked,Title,Familysize
0,892,3,1,3.0,7.8292,2,0,0
1,893,3,2,3.0,7.0000,0,1,1
2,894,2,1,4.0,9.6875,2,0,0
3,895,3,1,2.0,8.6625,0,0,0
4,896,3,2,2.0,12.2875,0,1,2


In [13]:
test_passengerid = test['PassengerId']
test = test.drop(['PassengerId'],axis=1)
x = train.drop(['Survived'], axis=1)
y = train['Survived']
xtrain,xtest, ytrain,ytest = train_test_split(x,
    y,test_size = 0.25, random_state = 123)

xtrain.shape, ytrain.shape, xtest.shape, ytest.shape

((668, 7), (668,), (223, 7), (223,))

In [14]:
# XGBoost shows the highest accuracy, so I'll use XGBoost model for test.
clf = xgb.XGBClassifier()
clf.fit(xtrain,ytrain)
test_prediction = clf.predict(xtest)  # Prediction with xtest
print('Test Accuracy : ', accuracy_score(test_prediction,ytest))

Test Accuracy :  0.8385650224215246


In [15]:
# Train all dataset in XGBoost and make a real prediciton on test
clf = xgb.XGBClassifier()
clf.fit(x,y)
score = cross_val_score(clf, x,y, cv=10, n_jobs=1, scoring='accuracy')
print(score)
print('Average Accuracy :' , round(np.mean(score)*100, 2))

[0.76666667 0.81111111 0.75280899 0.8988764  0.8988764  0.82022472
 0.84269663 0.76404494 0.82022472 0.82954545]
Average Accuracy : 82.05


In [16]:
# Prediciton by XGBoost

result = clf.predict(test)

result[0:10,]

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0])

In [17]:
import collections, numpy

collections.Counter(result)

submission= pd.DataFrame({
    "PassengerId" : test_passengerid,
    'Survived' : result
})

submission.to_csv('./submission/xgboost_turorial.csv',index=False)